In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install transformers pydub
!apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0bj7pu9_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0bj7pu9_
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=6f29c74bbd01efb51a787af033b82cc7b4a5221317a247ccfb60768bd940dfa3
  Stored in directory: /tmp/pip-ephem-wheel-cache-h6a2_boh/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import whisper
from transformers import pipeline
from pydub import AudioSegment


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [ ]:
# Whisper for transcription (small = fast, medium = accurate, large = best)
whisper_model = whisper.load_model("small")

# HuggingFace Summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


100%|███████████████████████████████████████| 461M/461M [00:18<00:00, 25.5MiB/s]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
from google.colab import files

print("📤 Upload a short audio file (mp3/wav, ~10-20s)")
uploaded = files.upload()

audio_path = list(uploaded.keys())[0]


📤 Upload a short audio file (mp3/wav, ~10-20s)


Saving Sunoy_Roy_Introduction.mp3 to Sunoy_Roy_Introduction.mp3


In [ ]:
# Convert to wav (Whisper prefers wav)
sound = AudioSegment.from_file(audio_path)
sound.export("temp.wav", format="wav")

# Transcribe
result = whisper_model.transcribe("temp.wav", language=None)  # auto language detection
transcript = result["text"]

print("🎙 Transcript:\n", transcript)


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🎙 Transcript:
  Hello, good afternoon. Myself Shinoi Roy and I am from Kolkata West Mahal. Today I am going to share a few moments which shaped me beyond my resume and which are not mentioned there. So one of the biggest part of my life has been football. This game has taught me so much in my life. And I was the captain of my college team. We went to play Inter-Drippalite Sports Meet. So as a captain it was my duty to manage my team on and off the field. It wasn't just about winning. It was keeping my team together and lifting their spirits in the dire situations even if we win or lose. So it also taught me to stay calm under pressure. So these experiences taught me the real meaning of teamwork and that I am more of a collaborative person rather than being solo as everything just is better together. So this experience really shaped who am I and these are the lessons I carry forward. And talking about curiosity, like even in the recent scanners I was really curious about how these large

In [ ]:
summary = summarizer(transcript, max_length=100, min_length=30, do_sample=False)
print("📝 Notes:\n", summary[0]['summary_text'])


📝 Notes:
 Shinoi Roy was the captain of his college team. He says football taught him the real meaning of teamwork. His curiosity led him to do more and more research on his own.


In [ ]:
user_pref = "Beginner"  # try Beginner / Intermediate / Advanced

if user_pref == "Beginner":
    prompt = "Explain in simple bullet points with examples: " + transcript
elif user_pref == "Advanced":
    prompt = "Summarize concisely with keywords and technical terms: " + transcript
else:
    prompt = "Summarize in 2-3 sentences: " + transcript

# Use summarizer again
personal_summary = summarizer(prompt, max_length=120, min_length=40, do_sample=False)
print("🎯 Personalized Notes:\n", personal_summary[0]['summary_text'])


🎯 Personalized Notes:
 Shinoi Roy was the captain of his college team. Football taught him the real meaning of teamwork. He is fascinated by the mountains and the terrains of Kolkata West Mahal. He was rejected for a job due to his lack of skills.
